# Кейс ЦБ РФ

## Подключение нужных библиотек

In [58]:
import numpy as np
import torch
from tqdm import tqdm

## Создаем Dataset для дальнейшей работы

In [45]:
class CbrDataset(torch.utils.data.Dataset):
    def __init__(self, inp_file, out_file):
        self.x_data = np.load(inp_file)
        self.y_data = np.load(out_file)
        
    def __len__(self):
        return len(self.x_data)
        
    def __getitem__(self, idx):
        inputs = torch.from_numpy(self.x_data[idx])
        outputs = torch.from_numpy(self.y_data[idx]) 
        return (torch.flatten(inputs), torch.flatten(outputs))  

## Создаем нужную нейросеть

In [47]:
class CbrNet(torch.nn.Module):
  def __init__(self):
    super(CbrNet, self).__init__()
    self.hid1 = torch.nn.Linear(600, 300)  
    self.hid2 = torch.nn.Linear(300, 100)
    self.oupt = torch.nn.Linear(100, 15)

    torch.nn.init.xavier_uniform_(self.hid1.weight) 
    torch.nn.init.zeros_(self.hid1.bias)
    torch.nn.init.xavier_uniform_(self.hid2.weight)
    torch.nn.init.zeros_(self.hid2.bias)
    torch.nn.init.xavier_uniform_(self.oupt.weight)
    torch.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = torch.tanh(self.hid1(x)) 
    z = torch.tanh(self.hid2(z))
    z = self.oupt(z)  
    return z


## Обучение модели

In [59]:
def train(model, ds, bs, lr, me, le):
    train_ldr = torch.utils.data.DataLoader(ds, batch_size=bs, shuffle=True)
    loss_func = torch.nn.L1Loss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in tqdm(range(0, me)): # RSF
        epoch_loss = 0.0  
        for (b_idx, batch) in enumerate(train_ldr):
            X = batch[0]
            X = X.to(torch.float32)
            X = X.to(device) # RSF
            y = batch[1]
            y = y.to(torch.float32)
            y = y.to(device) # RSF
            optimizer.zero_grad()
            oupt = model(X)
            loss_val = loss_func(oupt, y)  
            epoch_loss += loss_val.item()  
            loss_val.backward()  
            optimizer.step()     

        if epoch % le == 0:
            print(f"'эпоха = {epoch: 04d}  |  loss = {epoch_loss:0.4f}")
            torch.save(model, f"./results/model_{epoch:04d}")
    
    torch.save(model, f"./results/model_final.pt")
        

## Главная программа

In [49]:
# 0. Начало
print("\nРешение кейса ЦБ России с использованием PyTorch")
np.random.seed(42) 
torch.manual_seed(42) 

# Проверяем наличие GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используем устройство:', device)


Решение кейса ЦБ России с использованием PyTorch
Используем устройство: cuda


In [50]:
# 1. Создание объектов Dataset и DataLoader
print("\nЗагрузка обучающего Dataset")
train_input_file = f"./data/y_smp_train.npy"
train_output_file = f"./data/pars_smp_train.npy"
train_dataset = CbrDataset(train_input_file, train_output_file)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True) # RSF


Загрузка обучающего Dataset


In [51]:
data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())
# out: torch.Size([16, 3, 32, 32])
print(labels_batch.size())
# out: torch.Size([16])

#  test_file = ".\\Data\\boston_test.txt"
#  test_ds = BostonDataset(test_file)

torch.Size([16, 600])
torch.Size([16, 15])


In [52]:
# 2. Создание модели
print("\nСоздание 600-(300-100)-15 регрессионной нейросети")
net = CbrNet().to(device)
net.train()


Создание 600-(300-100)-15 регрессионной нейросети


CbrNet(
  (hid1): Linear(in_features=600, out_features=300, bias=True)
  (hid2): Linear(in_features=300, out_features=100, bias=True)
  (oupt): Linear(in_features=100, out_features=15, bias=True)
)

In [60]:
# 3. Обучаем модель
bs=10 # размер батча
lr=0.005 
me=50 # Максимальное количество эпох
le=10 # Точки выдачи отчета

print(f"\nbatch size = {bs}", end = " ")
print("функция потерь = L1Loss()", end = " ")
print("оптимизатор = Adam", end = " ")
print(f"learn rate = {lr}", end = " ")
print(f"количество эпох = {me}")

print("\nНачало обучения ")
train(net, train_dataset, bs, lr, me=50, le=10)
print("Завершение")


batch size = 10 функция потерь = L1Loss() оптимизатор = Adam learn rate = 0.005 количество эпох = 50

Начало обучения 


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

epoch =    0  |  loss = 21416.6048


 20%|███████████████▊                                                               | 10/50 [49:51<3:17:44, 296.61s/it]

epoch =   10  |  loss = 21310.0089


...

epoch =   30  |  loss = 21260.3964


 80%|███████████████████████████████████████████████████████████████▏               | 40/50 [3:21:28<49:21, 296.18s/it]

epoch =   40  |  loss = 21136.3213


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [4:11:07<00:00, 301.36s/it]


Завершение


In [ ]:
# 4. Точность модели
net.eval()
acc_train = accuracy(net, train_ds, 0.20)
print(f"\nAccuracy on train (within 0.20) = {acc_train:%0.4f}")
acc_test = accuracy(net, test_ds, 0.20)
print("Accuracy on test (within 0.20) = %0.4f " % acc_test)

# 5. Сохранение модели
torch.save(model, f".\results\model_final.pt")

# 6. Использование модели
  print("\nPredicting normalized (poverty, price) first train")
  print("Actual (poverty, price) = (0.0914, 0.2160) ")

  x = np.array([0.000273, 0.000, 0.0707, -1, 0.469,
    0.6421, 0.789, 0.049671, 0.02, 0.242, 0.178,
    0.39690], dtype=np.float32)
  x = T.tensor(x, dtype=T.float32)

  with T.no_grad():
    oupt = net(x)
  print("Predicted poverty price = %0.4f %0.4f " % \
    (oupt[0], oupt[1]))

  print("\nEnd demo ")


In [19]:
# 1. Создание объектов Dataset и DataLoader
print("Загрузка обучающего Dataset")
train_input_file = f"./data/y_smp_train.npy"
train_output_file = f"./data/pars_smp_train.npy"
train_dataset = CbrDataset(train_input_file, train_output_file)

Загрузка обучающего Dataset


In [20]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True) # RSF

data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())
# out: torch.Size([16, 3, 32, 32])
print(labels_batch.size())
# out: torch.Size([16])

torch.Size([1, 600])
torch.Size([1, 15])


In [21]:
data_batch

tensor([[-2.8332e-01,  4.7977e-01,  9.7841e-01,  7.0201e-01, -2.0094e-01,
          7.7660e-01,  1.2025e+00,  1.1255e+00,  1.3197e+00,  1.7523e-01,
          6.7688e-01,  1.2769e+00, -7.1684e-01,  5.0243e-01,  1.0748e+00,
          8.7993e-01,  1.7653e-01,  9.0398e-01,  3.7329e-01,  1.0904e+00,
          1.2568e+00, -1.1819e-01,  1.1853e-01,  8.3293e-01,  2.6479e-02,
          3.6586e-01,  7.9852e-01,  4.6848e-01, -1.3531e-01,  5.2652e-01,
          1.9251e-01,  1.1531e+00,  1.0780e+00, -2.8808e-01, -1.3341e-01,
          7.3784e-01, -2.9369e-01,  4.6119e-01,  7.7576e-01,  5.7129e-01,
         -8.7354e-02,  5.7750e-01,  3.8966e-01,  9.6757e-01,  1.0032e+00,
          3.1628e-01,  7.1965e-02,  7.0602e-01,  3.4976e-02,  7.0219e-01,
          8.0395e-01,  3.7736e-01, -2.6977e-01,  4.6718e-01,  5.0266e-01,
          4.6724e-01,  7.1826e-01,  1.5121e+00,  2.6587e-01,  7.1151e-01,
          5.4766e-02,  1.1167e+00,  1.2594e+00, -1.2524e-01, -2.4454e-01,
          5.8905e-01, -4.5405e-02,  8.

In [56]:
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [57]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
